# Inference
Get predictions for submission

In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import pretrainedmodels
import pandas as pd
import os

from tools import *


In [2]:
device = get_device()

Setting DEVICE:
	 MPS is available


## Importing label dataset

To run the full crossvalidation, remove .head(30000). This will multiply about by 20 the time required by this notebook.

In [3]:

# FOLDERS PATH
source_dir = 'histopathologic-cancer-detection/'
test_im_source_dir = source_dir+'/test'

# IMPORTING DATA
test_data = pd.read_csv(os.path.join(source_dir,'sample_submission.csv')) 



In [4]:
data_ids = list(test_data.id)
#data_labels = list(test_data.label)

## Loading model

In [5]:
model_source_dir = 'trained_model/'
model_name = 'final'

model_path =  model_source_dir+'/{}.model.pt'.format(model_name)


In [6]:
# Import pretrained model
base_model = pretrainedmodels.resnet34(num_classes=1000, 
                                    pretrained='imagenet').to(device) 
    
# Shape the model    
model = Net(base_model, 512).to(device)

# Load model from saved 
model.load_state_dict(torch.load(model_path))



/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

## Inference

In [7]:
model.eval() 
predictions = []

with torch.no_grad():    
    for imid in data_ids:
        imid = imid+'.tif'
        image = imread(os.path.join(test_im_source_dir, imid))
        image = cvtColor(image, COLOR_BGR2RGB)
        
        # TTA : augmentation
        images = produce_test_time_augmentation(image,number=8)
        
        output = model(torch.from_numpy(images).to(device, dtype=torch.float))
        
        # TTA : averaging the outputs
        output = output.mean()            

        predictions.append({'id': imid[:-4], 'label' : output.cpu().numpy()})

  


In [8]:

df = pd.DataFrame(predictions)

In [9]:
df.head()

,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0.0006543468
1,95596b92e5066c5c52466c90b69ff089b39f2737,0.0014875743
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0.0005637817
3,2c35657e312966e9294eac6841726ff3a748febf,0.00025511932
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0.0031613084


In [10]:
soglia = .5 #df.label.mean()

In [11]:
#df['id'] = df['id'].apply(lambda x: x[:-4])
df['label'] = df['label'].apply(lambda x: 1 if x > soglia else 0)


In [12]:
df.to_csv("submission.csv", index=False)